<a href="https://colab.research.google.com/github/strange-hawk/ml_nn/blob/master/sentiment_analysis_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/udacity/deep-learning.git

Cloning into 'deep-learning'...
remote: Enumerating objects: 1450, done.
remote: Total 1450 (delta 0), reused 0 (delta 0), pack-reused 1450
Receiving objects: 100% (1450/1450), 47.19 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (608/608), done.


In [2]:
import tensorflow as tf

In [0]:
!pip uninstall tensorflow

In [0]:
!pip install tensorflow

In [1]:
!pip install TFLearn

In [0]:
import pandas as pd

In [0]:
reviews = pd.read_csv('/content/deep-learning/sentiment-network/reviews.txt')

In [0]:
labels = pd.read_csv('/content/deep-learning/sentiment-network/labels.txt')

In [14]:
reviews.iloc[2]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t       airport    starts as a brand new luxury    pla...
Name: 2, dtype: object

In [0]:
from collections import Counter
total_counts = Counter()
for _,rows in reviews.iterrows():
  total_counts.update(rows[0].split(' '))

In [0]:
vocab = sorted(total_counts,key=total_counts.get,reverse=True)[:10000]

In [35]:
print(vocab[len(vocab)-1],total_counts[vocab[len(vocab)-1]])

fulfilled 30


In [0]:
word2index={}
for i,j in enumerate(vocab):
  word2index[j]=i

In [0]:
import numpy as np
def text2vector(review):
  review = str(review)
  inputt = np.zeros(len(vocab),dtype=np.int_)
  c=Counter()
  c.update(review.split(' '))
  for k,v in c.items():
    t = word2index.get(k,None)
    if t==None:
      continue
    else:
      inputt[t]=v
  return np.array(inputt)
  

In [64]:
text2vector('The tea is for a party to celebrate the movie so she has no time for a cake')[:65]
# type(reviews.iloc[0])

array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [0]:
word_vector = np.zeros((len(reviews),len(vocab)))
for i in range(len(reviews)):
  word_vector[i]=text2vector(reviews.iloc[i][0])

In [0]:
Y=(labels=='positive').astype('int')
Y.head()

In [80]:
import tflearn
from tflearn.data_utils import to_categorical


shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)
records=len(labels)
test_fraction=0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vector[train_split,:], to_categorical(Y.values[train_split, 0], 2)
testX, testY = word_vector[test_split,:], to_categorical(Y.values[test_split, 0], 2)

In [83]:
len(shuffle)

24999

In [0]:
import tensorflow as tf


In [0]:
# network building

def build_model():
  tf.reset_default_graph()
  net = tflearn.input_data([None,10000])
  # hidden
  net = tflearn.fully_connected(net,200,activation='ReLU')
  net = tflearn.fully_connected(net,25,activation='ReLU')
  # output
  net = tflearn.fully_connected(net,2,activation='softmax')
  net = tflearn.regression(net,optimizer='sgd',learning_rate=0.1,loss='categorical_crossentropy')
  model = tflearn.DNN(net)
  return model

In [0]:
model = build_model()

In [92]:
model.fit(trainX,trainY,n_epoch=100,show_metric=True,batch_size=128,validation_batch_size=0.1)

Training Step: 17599  | total loss: 0.27892 | time: 6.036s
| SGD | epoch: 100 | loss: 0.27892 - acc: 0.8679 -- iter: 22400/22499
Training Step: 17600  | total loss: 0.28338 | time: 6.070s
| SGD | epoch: 100 | loss: 0.28338 - acc: 0.8709 -- iter: 22499/22499
--


In [93]:
predictions = (np.array(model.predict(testX))[:,0]>0.5).astype('int')
test_accuracy = np.mean(predictions==testY[:,0],axis=0)
print("test_accuracy",test_accuracy)

test_accuracy 0.8488


In [0]:
def test_sentence(review):
  positive_prob = model.predict([text2vector(review)])
  # print(positive_prob)
  if positive_prob[0][0]>0.5:
    print('POSITIVE',positive_prob[0][0])
  else:
    print('NEGATIVE',positive_prob[0][1])

In [106]:
test_sentence("It's amazing anyone could be talented enough to make something this spectacularly awful")

POSITIVE 0.998287
